# Create Claims Detached Prompt Template


This notebook should be run using with Runtime 22.2 & Python 3.10 or greater runtime environment, if you are viewing this in Watson Studio, and do not see Python 3.10.x or greater in the upper right corner of your screen, please update the runtime now. 

The notebook will create a RAG prompt template asset in a given project.


Note : User can search for `EDIT THIS` and fill the inputs needed.

## Prerequisite

* It requires service credentials for IBM Watson OpenScale/watsonx
* Requires the ID of project in which you want to create the prompt template asset.

### Contents

- [Setup](#settingup)
- [Create Detached Prompt template](#prompt)


## Setup <a name="settingup"></a>

In [ ]:
!pip install --upgrade datasets==2.10.0 --no-cache | tail -n 1
!pip install --upgrade evaluate --no-cache | tail -n 1
#!pip install --upgrade --extra-index-url https://test.pypi.org/simple/ ibm-aigov-facts-client | tail -n 1
!pip install --upgrade ibm-aigov-facts-client | tail -n 1
!pip install --upgrade ibm-watson-openscale | tail -n 1
!pip install --upgrade matplotlib | tail -n 1
!pip install --upgrade pydantic==1.10.11 --no-cache | tail -n 1
!pip install --upgrade sacrebleu --no-cache | tail -n 1
!pip install --upgrade sacremoses --no-cache | tail -n 1
!pip install --upgrade textstat --no-cache | tail -n 1
!pip install --upgrade ibm_aigov_facts_client --no-cache | tail -n 1
!pip install --upgrade openai --no-cache | tail -n 1 
!pip install --upgrade azure.identity --no-cache | tail -n 1

Note: you may need to restart the kernel to use updated packages.

### Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/watson-openscale).

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

**NOTE:** You can also get OpenScale `API_KEY` using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

In [8]:
use_cpd = False

if use_cpd:
    CPD_URL = "https://cpd-cpd-instance.apps.gpu01.pok-lb.techzone.ibm.com/"
    CPD_USERNAME = "EDIT THIS"
    CPD_APIKEY = "EDIT THIS"
    WOS_SERVICE_INSTANCE_ID = "None" # If None, default instance would be used
else:
    IAM_URL = "https://iam.cloud.ibm.com"
    DATAPLATFORM_URL = "https://api.dataplatform.cloud.ibm.com"
    SERVICE_URL = "https://aiopenscale.cloud.ibm.com"
    CLOUD_API_KEY = "EDIT THIS" # YOUR_CLOUD_API_KEY
    

## Set the project ID

In order to set up a development type subscription, the PTA must be within the project. Please supply the project ID where the PTA needs to be created.

In [9]:
PROJECT_ID = "EDIT THIS" # YOUR_PROJECT_ID

## Function to create the access token

This function generates an IAM access token using the provided credentials. The API calls for creating and scoring prompt template assets utilize the token generated by this function.

In [10]:
import requests, json
def generate_access_token():
    headers={}
    if not use_cpd: 
        headers["Content-Type"] = "application/x-www-form-urlencoded"
        headers["Accept"] = "application/json"
        data = {
            "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
            "apikey": CLOUD_API_KEY,
            "response_type": "cloud_iam"
        }
        response = requests.post(IAM_URL + "/identity/token", data=data, headers=headers)
        json_data = response.json()
        iam_access_token = json_data["access_token"]
    else:
        headers["Content-Type"] = "application/json"
        headers["Accept"] = "application/json"
        data = {
            "username": CPD_USERNAME,
            "api_key": CPD_APIKEY
        }
        data = json.dumps(data).encode("utf-8")
        url = CPD_URL + "/icp4d-api/v1/authorize"
        response = requests.post(url=url, data=data, headers=headers, verify=False)
        json_data = response.json()
        iam_access_token = json_data["token"]
        
    return iam_access_token

iam_access_token = generate_access_token()

# Create Detached Prompt template <a name="prompt"></a>

Create a detached prompt template for a RAG task

In [11]:
from ibm_aigov_facts_client import AIGovFactsClient

if not use_cpd:
    facts_client = AIGovFactsClient(
        api_key=CLOUD_API_KEY,
        container_id=PROJECT_ID,
        container_type="project",
        disable_tracing=True
    )
else:
    from ibm_aigov_facts_client import CloudPakforDataConfig
    creds=CloudPakforDataConfig(
        service_url=CPD_URL,
        username=CPD_USERNAME,
        api_key=CPD_APIKEY
    )
    facts_client = AIGovFactsClient(
        cloud_pak_for_data_configs=creds,
        container_id=PROJECT_ID,
        container_type="project",
        disable_tracing=True
    )

In [ ]:
from ibm_aigov_facts_client import DetachedPromptTemplate, PromptTemplate

detached_information = DetachedPromptTemplate(
    prompt_id="detached_prompt",
    model_id="openai-gpt-35-turbo",
    model_provider="Azure OpenAI",
    model_name="openai-gpt-35-turbo",
    model_url="https://azureml-openai-americas-1.openai.azure.com/",
    prompt_url="prompt_url",
    prompt_additional_info={"model_owner": "openai"}
)

task_id = "summarization"
name = "Insurance claim summarization - gpt3.5"
description = "Detached prompt template using Azure OpenAI gpt-35-turbo model that interprets data from the wikipedia article on the Elevator."
model_id = "openai-gpt-35-turbo"

# define parameters for PromptTemplate
prompt_variables = {"input":""}
input = "You are an insurance agent tasked to assess insurance claims. Summarize the following insurance claim input. Focus on the car and the damage. Make the summary at least 3 sentences long.\n\nInput: {input}\nOutput:"
input_prefix= ""
output_prefix= ""

prompt_template = PromptTemplate(
    input=input,
    prompt_variables=prompt_variables,
    input_prefix=input_prefix,
    output_prefix=output_prefix
)

pta_details = facts_client.assets.create_detached_prompt(
    model_id=model_id,
    task_id=task_id,
    name=name,
    description=description,
    prompt_details=prompt_template,
    detached_information=detached_information
)
project_pta_id = pta_details.to_dict()["asset_id"]

## Congratulations!

You have finished creating a detached prompt template! You can now navigate to the detached prompt template asset in your project and evaluate and track to a use case.